In [ ]:
!pip install matplotlib

In [ ]:
import syft as sy
%matplotlib inline
duet = sy.join_duet(loopback=True)

In [ ]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        
        self.lin =self.torch_ref.nn.Linear(6,10)
    def forward(self, x):
        x = self.lin(x)
        return x

In [ ]:
import torch
model1 = SyNet(torch)
model1_ptr = model1.send(duet)

In [ ]:
opt1 = duet.torch.optim.SGD(params=model1_ptr.parameters(),lr=0.00001)

In [ ]:
duet.store.pandas

In [ ]:
data_pointer = duet.store[0]

In [ ]:
import numpy as np

In [ ]:

model2 = torch.nn.Linear(10,6)
opt2 = torch.optim.SGD(params=model2.parameters(), lr=0.00001)

target = torch.tensor([[17.62,17.73,17.81,17.95,18.06,18.09],[18.17,18.23,18.29,18.3,18.07,18.16],
                      [18.19,18.24,18.28,18.31,18.38,18.42],[18.49,18.57,18.59,18.62,18.64,18.69],
                      [18.73,18.77,18.86,19.17,19.23,19.36],[19.45,19.58,19.63,19.75,19.87,19.89],
                      [19.95,20.06,20.09,20.13,20.18,20.24],[20.27,20.31,20.39,20.42,20.48,20.56],
                      [20.67,20.73,20.79,20.84,20.89,20.93],[20.98,21.06,21.09,21.15,21.19,21.24]])
target.shape
#target.type

In [ ]:
%%time
import matplotlib.pyplot as plt
#epoch = 200
losses = []
predictions = []
for iter in range(250):
    opt1.zero_grad()
    opt2.zero_grad()
    
    activation_ptr = model1_ptr(data_pointer)
    activation = activation_ptr.clone().get(request_block=True)
    
    pred = model2(activation)
    
    #target = target.detach().numpy()
    loss = ((pred - target)**2).sum()
   
    loss.backward()
    
    grad_ptr = activation.grad.clone().send(duet)
    activation_ptr.backward(grad_ptr)
    
    opt1.step()
    opt2.step()
    losses.append(loss.tolist())
    predictions.append(pred.tolist())
    
    print("losses: ",losses)
    print('Predictions =',pred)


In [ ]:
%%time
import matplotlib.pyplot as plt
#%matplotlib inline
#pred = pred.detach().numpy()
#pred = pred.numpy()
#loss = loss.detach().numpy()
# print("lost",losses)
# print("list",losses[0].tolist())
# print("pred", predictions[0].tolist())
# print("pred", predictions[0].ravel().tolist())
#model2 = model2.detach().numpy()
#target = target.detach().numpy()
plt.title('Diameter of Bulb-Flow Hydroponics')
plt.plot(losses,color='brown')
plt.ylabel("Loss")
plt.xlabel("epoch")
plt.show()